In [1]:
# initial imports

import h5py
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import pandas as pd

import torchvision
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim


import matplotlib as mpl
from matplotlib import pyplot as plt

import os
import re

In [2]:
# Set parameters, load data
year = '2024'
month = '12'
day = '05'

monkey = 'Bourgeois'

start_bin = -.1
end_bin = .301
blank_or_nonblank = 'blank'

pathname = f'/Users/parsatalaie/Desktop/Marmoset Datasets/{year}{month}{day}_all_psth.h5'

f = h5py.File(pathname)
list(f.keys())

['data',
 'site_coordinates',
 'stim_indices',
 'trial_params',
 'trial_params_short']

In [3]:
# Add path to and import mkturk analysis tools

from sys import path

print(path)

path.append('/Users/parsatalaie/Desktop/Issa Data')

from data_analysis_tools_mkTurk.utils_meta import get_recording_path
from data_analysis_tools_mkTurk.general import df_2_psth_mat
from data_analysis_tools_mkTurk.IO import ch_dicts_2_h5, h5_2_trial_df, h5_2_df
from data_analysis_tools_mkTurk.npix import chs_meta_2_site_coords

['/Users/parsatalaie/Desktop/Marmoset Data Analysis', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']


/Users/parsatalaie/Desktop/Issa Data/data_analysis_tools_mkTurk/general.py:23: UserWarning: Failed to import analysis_metadata module.
  warnings.warn('Failed to import analysis_metadata module.')
/Users/parsatalaie/Desktop/Issa Data/data_analysis_tools_mkTurk/IO.py:21: UserWarning: Failed to import analysis_metadata module.
  warnings.warn('Failed to import analysis_metadata module.')


In [4]:
# Create, print trial_params

trial_params = h5_2_trial_df(pathname)

/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


In [5]:
# List scenefiles

trial_params['scenefile'].unique()

array(['/mkturkfiles/scenebags/West/BlankStim_300ms.json',
       '/mkturkfiles/scenebags/West/neural_stim_4_0ABCDEFGHIJ.json',
       '/mkturkfiles/scenebags/West/20231025_Rust_NaturalImages300_300ms.json',
       '/mkturkfiles/scenebags/West/20231025_Var6vbslir_set0_im151_neptune_dur300ms_lab_updated.json',
       '/mkturkfiles/scenebags/West/20231025_Var6vbslir_set0_im151_elias_dur300ms_lab_updated.json'],
      dtype=object)

In [6]:
# Rust
scenefiles = ['/mkturkfiles/scenebags/West/20231025_Rust_NaturalImages300_300ms.json']

In [6]:
# Objaverse
scenefiles = ['/mkturkfiles/scenebags/West/neural_stim_4_0ABCDEFGHIJ.json']

In [7]:
# HvM
scenefiles = ['/mkturkfiles/scenebags/West/hvm10_table_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_elephant_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_dog_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_bear_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_chair_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_car_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_turtle_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_plane_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_apple_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_head_45_20240906.json']

In [7]:
# Select stimulus presentations associated with requested scenefiles:
filter = trial_params.scenefile.isin(scenefiles)
rust_trials = trial_params[filter]
array_filter = np.array(rust_trials[['trial_num', 'rsvp_num']])

# Read spike count data from HDF5 for requested trials:
time_window = [start_bin, end_bin] # Beginning and end of peristimulus time window for each stim, relative to trigger in seconds
rust_data = h5_2_df(pathname, trials=array_filter, time_window=time_window)

# Sort rust_data to match trial_params, rust_trials
rust_data.sort_index(inplace=True)

Fetching trial parameters...
... done (0.022207021713256836 sec).
inds_df.shape = (900, 2)
Pre-fetching PSTHs from HDF5...


/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


... done.
Duration=0.014338382085164388 minutes
Fancy slicing numpy array...
... done.
Duration=0.0014680027961730957 minutes


In [8]:
# Create + apply mask to remove nans
mask = rust_data.apply(lambda x : np.all(np.isnan(x.psth)), axis=1)
final_df = rust_data[-mask]
final_df.shape

(731, 12)

In [9]:
# Convert psth to spike matrix
final_spike_arr = np.array(list(final_df.psth))
print(final_spike_arr.shape)

# Check for remaining nan
print(f'Nans remaining after removal: {np.isnan(final_spike_arr).sum()}')

(731, 384, 40)
Nans remaining after removal: 0


In [10]:
# Create avg_spikes
avg_spikes = np.mean(final_spike_arr, axis=2)
avg_spikes.shape

(731, 384)

In [43]:
# Objaverse pathlist
locker_prefix = '/Volumes/'
path_series = final_df['img_full_path'].str[16:].apply(lambda x : locker_prefix + x)
natimg_path_list = path_series.tolist()
len(natimg_path_list)

731

In [142]:
# Rust pathlist
path_series = final_df['stim_idx'].apply(lambda x : 
                                           f'/Users/parsatalaie/Downloads/rust_natimgs/Nat300_{x+1}.png')
natimg_path_list = path_series.tolist()
len(natimg_path_list)

3364

In [ ]:
# HvM pathlist without img_full_path column

# get spliced scenefile (for local pathing) (Check out warning if things go wrong...)
start_idx = 28
final_df['spliced_scenefile'] = final_df['scenefile'].str.removesuffix('.json').str[start_idx:]

# Generate img_full_path
hvm_prefix = '/Users/parsatalaie/Desktop/Marmoset Datasets/hvm10/'

def png_path_end_int(path):
    match = re.search(r'(\d+)\.png$', path)
    if match:
        return int(match.group(1))  # Returns "123" as string
    else:
        return None

for scenefile in final_df['spliced_scenefile'].unique():
    sfile_path = hvm_prefix + scenefile
    img_list = os.listdir(sfile_path)
    # Note: there is one .json in each scenefile, which is mapped to None in dict
    idx_to_path = {png_path_end_int(img):'/'.join([sfile_path, img]) for img in img_list}
    scene_mask = final_df['spliced_scenefile'] == scenefile
    final_df.loc[scene_mask, 'img_full_path'] = final_df.loc[scene_mask, 'stim_idx'].map(idx_to_path) 

natimg_path_list = final_df['img_full_path'].tolist()
len(natimg_path_list)

In [ ]:
# NOTE: Can use 57: + strip .png to double check that indices are aligned

In [12]:
# HvM pathlist using img_full_path
start_idx = 51
alter_hvm_path = lambda hvm_path : '/Users/parsatalaie/Desktop/Marmoset Datasets/' + hvm_path[start_idx:]
natimg_paths = final_df['img_full_path'].apply(alter_hvm_path)
natimg_path_list = natimg_paths.tolist()
len(natimg_path_list)

TypeError: 'NoneType' object is not subscriptable

In [44]:
# Transpose spikes to match formatting of old data
avg_spikes = avg_spikes.transpose()
avg_spikes.shape

(384, 731)

In [104]:
# Save avg_spikes
np.save(f'./{year}_{month}_{day}/marm_avg_spikes_{year}{month}{day}', avg_spikes)

In [105]:
# pickle it

import pickle

with open(f'./{year}_{month}_{day}/natimg_path_list_{year}{month}{day}', 'wb') as fp:   #Pickling
    pickle.dump(natimg_path_list, fp)

In [ ]:
'''
SHR
'''

In [47]:
from scipy.stats import zscore

marm_avg_spikes_t = avg_spikes
marm_avg_spikes = zscore(np.transpose(marm_avg_spikes_t.astype('float32')), axis=1)

In [48]:
# make new indices for SHR
ch_tot = marm_avg_spikes.shape[1]
stim_tot = marm_avg_spikes.shape[0]
indices = np.arange(stim_tot)

In [49]:
# get max_reps

trial_counts = []

for path in set(natimg_path_list):
    trial_counts.append(natimg_path_list.count(path))

max_reps = max(trial_counts)

min_repeat = 2

In [50]:
# HVM: Get demon matrix for each neuron

many_path_list = [path for path in natimg_path_list 
                      if natimg_path_list.count(path) >= min_repeat]
img_count = len(set(many_path_list))
sh_data = np.zeros(ch_tot)
img_spikes_tot = np.zeros((ch_tot, max_reps, img_count))

for ch in range(ch_tot):
    neuron = marm_avg_spikes[:, ch]
    count_arr = np.zeros((max_reps, img_count))
    count_arr[:] = np.nan
    for i, path in enumerate(set(many_path_list)):
        idx_list = [i for i, j in enumerate(natimg_path_list) if j == path]
        # note: original mistake, ^ should take natimg_path_list as arg
        for rep, idx in enumerate(idx_list):
            count_arr[rep, i] = neuron[idx]
    img_spikes_tot[ch] = count_arr
    neuron_1, neuron_2 = count_arr[::2], count_arr[1::2]
    neuron_1_avg = np.nanmean(neuron_1, axis=0)
    neuron_2_avg = np.nanmean(neuron_2, axis=0)
    sh, _ = stats.pearsonr(neuron_1_avg, neuron_2_avg)
    sh_data[ch] = sh

In [52]:
sh_limit = .3

# Get good idx
good_idx = np.where(sh_data >= sh_limit)[0]

# get good sh, exp
good_sh = sh_data[good_idx]

good_ch_tot = good_idx.shape[0]

print(f'Number of good neurons: {good_ch_tot}')

Number of good neurons: 297


In [ ]:
'''
Check if issue is filtering of df: start with all trials
'''

In [41]:
# h5_2_df the whole file (same time_window)

time_window = [start_bin, end_bin]
all_data = h5_2_df(pathname, time_window=time_window)

Fetching trial parameters...
... done (0.02410578727722168 sec).
Pre-fetching PSTHs from HDF5...


/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


... done.
Duration=0.01466134786605835 minutes
Fancy slicing numpy array...
... done.
Duration=0.01987988551457723 minutes


In [42]:
# Apply masks

nan_mask = all_data.apply(lambda x : np.all(np.isnan(x.psth)), axis=1)
fix_all_data = all_data[-nan_mask]
scenefile_mask = fix_all_data['scenefile'] == '/mkturkfiles/scenebags/West/20231025_Rust_NaturalImages300_300ms.json'
final_all_data = fix_all_data[scenefile_mask]
final_all_data.shape

(3364, 11)

In [43]:
# Convert psth to spike matrix
final_spike_arr = np.array(list(final_all_data.psth))
final_spike_arr.shape

# Check for remaining nan
np.isnan(final_spike_arr).sum()

# Create avg_spikes
avg_spikes = np.mean(final_spike_arr, axis=2)
print(avg_spikes.shape)

# Create natimg_path_list
path_series = final_all_data['stim_idx'].apply(lambda x : 
                                           f'/Users/parsatalaie/Downloads/rust_natimgs/Nat300_{x+1}.png')
natimg_path_list = path_series.tolist()
len(natimg_path_list)

(3364, 384)


3364

In [44]:
# Check if equivalent

try:
    pd.testing.assert_frame_equal(final_all_data.sort_index(), final_df)
    print("DataFrames are identical")
except AssertionError:
    print("DataFrames are different")

NameError: name 'final_df' is not defined

In [30]:
# Dan new ch_depth
zero_coords = pd.read_hdf(pathname, 'zero_coordinates')
imro_tbl = pd.read_hdf(pathname, 'imro_table')

zero_coords['monkey'] = monkey
zero_coords['date'] = f'{year}{month}{day}'
imro_tbl['monkey'] = monkey
imro_tbl['date'] = f'{year}{month}{day}'

imro_tbl['ch_idx_glx'] = imro_tbl.index

chs_meta_df = chs_meta_2_site_coords(zero_coords, imro_tbl)

ch_depths = np.array(chs_meta_df['depth'])

/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


In [29]:
# Save ch_depths

np.save(f'./{year}_{month}_{day}/ch_depth_{year}{month}{day}', ch_depths)

In [106]:
# Create new_ch_depth

coords = f['site_coordinates']
depth_ch_idx = coords['axis1'][:]

new_ch_depth = np.zeros((384))

for i in range(384):
    idx = depth_ch_idx[i]
    new_ch_depth[i] = coords['block0_values'][:][:, 3][idx]

KeyError: "Unable to open object (object 'site_coordinates' doesn't exist)"

In [8]:
# Save list of ch depths

np.save(f'./{year}_{month}_{day}/ch_depth_{year}{month}{day}', new_ch_depth)